In [1]:
from arcgis.gis import GIS
import getpass

In [2]:
arcgis_org_url="https://foretify.maps.arcgis.com/"
username= "foretify_proton"
password='Zion2023!!'

In [3]:
gis = GIS(arcgis_org_url, username, password, verify_cert=False)
print("Successfully logged in as: " + gis.properties.user.username)

Setting `verify_cert` to False is a security risk, use at your own risk.


Successfully logged in as: foretify_proton
